## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
cells_meta_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/cells_meta.tsv"
outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_clone_btwnvars_RNA/"
sample_names = "preB,postB"

#cells_meta_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/v2_MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/thr__0.01_rt__10/cells_meta.tsv"
#outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/v2_MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/thr__0.01_rt__10/annotation_clones/de_clone_btwnvars_RNA/minPct_0_logfc0_minCells1_pthresh0.1_pthresh_0.1"
min_cells = 1 

# Parameters
ncells_thresh = 10
minPct = 0.00
logfcthresh = 0.1
test.use="wilcox"
latent.vars = "NULL"
p_thresh = 0.1 
top_de = 3
ids.reversed = T

# Parallel processing
cores = 4
# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"


In [2]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")
library(dplyr)
library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required pa

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [3]:
source(utils_script)

In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "preB"  "postB"

In [5]:
dir.create(outdir)

In [6]:
cells_meta <- read.table(cells_meta_f, sep="\t", header = T, row.names="cell")
cells_meta 

#cells_meta
ids.reversed = T

if (ids.reversed){
    print('here')
    split.names <- stringr::str_split(rownames(cells_meta), "_")
    rownames(cells_meta) = lapply(split.names, function(x){paste0(x[2], "_", x[1])})
}
cells_meta = cells_meta[, order(colSums(-cells_meta))]
head(cells_meta)

Warning message in file(file, "rt"):
"cannot open file '/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/mt_clones/cells_meta.tsv': No such file or directory"


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
se

### Subset to just in cells_meta

In [ ]:
se$inClones = 0
se[["inClones"]][(colnames(se) %in% row.names(cells_meta)), "inClones"] = 1

sum(se[["inClones"]])/dim(se[[]][1])

se.filt <- subset(se, subset = inClones == 1 )

In [ ]:
se.filt <- AddMetaData(se.filt, cells_meta)

In [ ]:
se

In [ ]:
se.filt

In [ ]:
source(utils_script)
tmp <- get.clone.sizes(se.filt)
counts.conds.norm <- tmp[["counts.norm"]]
clone.sizes <- tmp[["clone.size"]]
head(clone.sizes)

In [ ]:
dim(se.filt[[]])

## plot each clone pre and post 

In [ ]:
mt.vars <- colnames(cells_meta)
mt.vars

In [ ]:
se.filt[[]]

In [ ]:
for (mt in (mt.vars)){
    print(mt)    
    expr <- FetchData(se.filt, vars = mt)
    curr.se <- se.filt[, which(expr == 1)]
    #curr.se
    p <- DimPlot(se.filt , group.by = "condition", split.by=mt)
    p2 <- DimPlot(curr.se , split.by="condition")
    curr.count = sum(se.filt[[mt]])
    ggsave(file.path(outdir, paste("count", curr.count,mt, "groupCond.png", sep="_")), p)
    ggsave(file.path(outdir, paste("count", curr.count,mt, "clrCond.png", sep="_")), p2)
}

In [ ]:
outdir

# Stimulus specific response in each clone

A. with variant vs without
B. top variants against top variants

In [ ]:
#    expr <- FetchData(se.filt, vars = mt)
    #curr.se <- se.filt[, which(expr == 1)]
    #curr.se

In [ ]:
all.pvals <- data.frame(matrix(nrow=dim(se.filt)[1],ncol=0),
                          row.names=row.names(se.filt))
all.names.sig <- c()

# #sample_names

for (curr.clone in mt.vars){
    print(curr.clone)
    clone.ids <- sort(unique(se.filt$name))
    se.filt$clone.stim <- paste(se.filt[[curr.clone]][,curr.clone], se.filt$condition,  sep = "_")

    # se$clone <- se$name
    Idents(se.filt) <- "clone.stim"

    ident.1 = paste0("0", "_", sample_names[[1]]) 
    ident.2 = paste0("1", "_", sample_names[[2]])
    #try({
        print(curr.clone)
        curr.name <-paste0("clone_", curr.clone)
        curr.clone.outdir <- file.path(outdir, paste0("clone_", curr.clone))
        dir.create(curr.clone.outdir)

        curr.sig <- find.markers.and.plot(se.filt, ident.1, ident.2, curr.clone.outdir, 
                                          curr.name=curr.clone, assay="RNA", logfcthresh=logfcthresh,
                                          min.pct=minPct, p.thresh=p_thresh, mean.fxn=NULL, fc.name=NULL)
        names.sig <- rownames(curr.sig)
        all.names.sig <- c(all.names.sig, head(names.sig, top_de))
        all.pvals[names.sig,curr.clone] <- curr.sig$p_val_adj_BH        
        #})
    }



all.pvals[rowSums(is.na(all.pvals)) != ncol(all.pvals), ]




In [ ]:
# ## DE sig summarize

# all.names.sig

# ### a) top 3 significant results from each comparison

# all.names.sig <- unique(all.names.sig)
# de.plots(se, all.names.sig, btwnConds_inClone_sepDonors.outdir, curr.name="allDonors.allClusters", max.size=15)

# dot <- DotPlot(se, scale=FALSE,
#                features = head(all.names.sig,20), 
#                cluster.idents=T) + RotatedAxis()
# dot
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.top3de.png"))
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.top3de.pdf"))



# ## b) averaged top p-values (only average ones that were DE)

# sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
# dot <- DotPlot(se, scale=T,
#                features = names(head(sig.all.ordered,20)), 
#                cluster.idents=T) + RotatedAxis()
# dot
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.pvalsOrdered.png"))
# ggsave(file.path(btwnConds_inClone_sepDonors.outdir, "allDonors.allClusters.pvalsOrdered.pdf"))
# #de.plots(se, names(sig.all.ordered), outdir, curr.name="allDonors.allClusters.pvalsOrdered", max.size=20)

# sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
# dot <- DotPlot(se, scale=F,
#                features = head(names(sig.all.ordered),20), 
#                cluster.idents=T) + RotatedAxis()
# dot


In [ ]:
sessionInfo()

In [ ]:
outdir